This notebook focus on the implementation of the following publication with

* [Rate-conforming Sub-band Allocation for In-factory
Subnetworks: A Deep Neural Network Approach](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=10597067)

The model and hyperparameters will be modified to work with our simulation resutls.

In [1]:
# simple data manipulation
import numpy  as np
import pandas as pd

# deep learning framework
import torch
import torch.nn as nn
import torch.nn.functional as F

# logging results and progress
import wandb
import tqdm

# remove warnings (remove deprecated warnings)
import warnings
warnings.simplefilter('ignore')

# visualization of results
import matplotlib.pyplot as plt
from   matplotlib.ticker import MaxNLocator
import seaborn           as sns

# Graph Algorithms.
import networkx as nx

# Google Colab (many lines are removed)
import os
import zipfile
# from google.colab import drive
# from distutils.dir_util import copy_tree

# wheter we are using colab or not
COLAB: bool = False
if not COLAB: os.chdir('..')

# Simulation Settings
from data.sim_config import SimConfig
config = SimConfig(0, './config.ini')
config

Simulation Parameters: 

|                      name |                     value |
---------------------------------------------------------
|        num_of_subnetworks |                   20.0000 |
|              n_subchannel |                    4.0000 |
|             deploy_length |                   20.0000 |
|             subnet_radius |                    1.0000 |
|                      minD |                    0.8000 |
|               minDistance |                    2.0000 |
|                 bandwidth |            100000000.0000 |
|              ch_bandwidth |             25000000.0000 |
|                        fc |           6000000000.0000 |
|                    lambdA |                    0.0500 |
|                  clutType |                     dense |
|                  clutSize |                    2.0000 |
|                  clutDens |                    0.6000 |
|                   shadStd |                    7.2000 |
|                 max_power |                  

## Simulations and Information

Thanks to the given scripts, we can load a group of generated simulations. They don't have any solutions (neither approximations).

In [2]:
# Moung Google Drive Code
if COLAB:
    # drive.mount('/content/drive')

    # Move Simulations to avoid cluttering the drive folder
    # if not os.path.exists('/content/simulations'):
    #   os.mkdir('/content/simulations')

    # if list(os.listdir('/content/simulations')) == []:
    #   copy_tree('/content/drive/MyDrive/TFM/simulations', '/content/simulations')

    # unzip all simulations
    # print("Name of the already simulated data: \n", )
    for zip_file in os.listdir('/content/simulations'):
        if zip_file.endswith('.zip'):
            print(" ----> " + zip_file)
            with zipfile.ZipFile("/content/simulations/" + zip_file, 'r') as zip_ref:
                zip_ref.extractall('/content/simulations/')

    SIMULATIONS_PATH: str = "/content/simulations"
else:
    if not os.path.exists('./data/simulations'):
        os.mkdir('./data/simulations')
    for zip_file in os.listdir('data'):
        if zip_file.endswith('.zip'):
            print(" ----> " + zip_file)
            with zipfile.ZipFile("./data/" + zip_file, 'r') as zip_ref:
                zip_ref.extractall('./data/simulations')
    SIMULATIONS_PATH: str = "./data/simulations"

## Generation of Approximate Allocations and Power

In [3]:
loc = np.load(SIMULATIONS_PATH + '/Location_mat.npy')
cmg = np.load(SIMULATIONS_PATH + '/Channel_matrix_gain.npy')

# for this test, we only consider some cases
loc, cmg = loc[:100], cmg[:100]
shape    = lambda s: " x".join([f"{d:3d}" for d in s])
print(f"location matrix shape: {shape(loc.shape)}\nchannel  matrix shape: {shape(cmg.shape)}")

location matrix shape: 100 x 20 x  2
channel  matrix shape: 100 x  4 x 20 x 20


To generate a list of initial approximations for training, we can apply [SISA](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=10118695) (Sequential Iterative Sub-band Allocation). This method allows us to generate almost-optimal approximations.

To apply this method, we need to consider that two things:

* the calculations require setting an initial transmission power. As  this is one of the proposed task, we can fix the value or setting other.

* The results are a baseline. ¿Should we remove it from the table?

For the initial power settings, we will fix a value, it is not necessary to know the value as it disappears in the calculations at the start. 
The algorithm will be iterated over 20 iterations.

In [15]:
def weighted_interference_matrix(channel_gain: np.ndarray) -> np.ndarray:
    """
    Computes the weighted interference matrix W based on the channel gain matrix.

    Parameters:
    - channel_gain (np.ndarray): A (K, N, N) array representing the channel gains 
      between different nodes and subnetworks.

    Returns:
    - W (np.ndarray): A (K, N, N) matrix where W[k, i, j] represents the 
      interference weight from node i to node j in subnetwork k.
    """
    # Compute direct channel gains and normalize interference
    Hd = np.expand_dims(np.diagonal(channel_gain, 0, 1, 2), 2)
    W  = np.where(Hd > 0, channel_gain / Hd , 0)
    
    #  remove self-interference
    for k in range(W.shape[0]): np.fill_diagonal(W[k], 0)

    return W

def sisa_algoritm(channel_gain: np.ndarray, max_iter: int = 20) -> tuple[np.ndarray, np.ndarray]:
    """
    Implements the Sequential Iterative Subband Allocation (SISA) Algorithm

    The algorithm iteratively assigns each subnetwork to a subband that minimizes the sum weighted
    interference of the whole network while ensuring a fair allocation.

    Args:
        - channel_gain (np.ndarray): A (K, N, N) array representing the channel gains.
        - max_iter (int): The maximum number of iterations for optimization. Defaults to 20 iterations.

    Returns:
        - A (np.ndarray): An (N, ) array where A[n] represents the assignment of the subnetwork n to the subband k.
        - F (np.ndarray): An (N x max_iter) array with a list of values of the sum interference of each step. 
    
    Reference:
        - [Advanced Frequency Resource Allocation for Industrial Wireless Control in 6G subnetworks](https://ieeexplore.ieee.org/document/10118695) 
    """
    K, N, _ = channel_gain.shape
    
    # inititalize inputs
    A = np.zeros((N)   , dtype=int) # A : N -> K
    B = np.zeros((K, N), dtype=int) # B_k : {n \in N : A(n) = k}
    
    # first all networks are assigned to the same subband
    B[0, :] = 1

    W = weighted_interference_matrix(channel_gain)
    total_interference = []

    # procedure
    for _ in range(1, max_iter + 1):
        for n in range(N):
            # 1. compute iteration number : not required
            # 2. compute w_k (d) for all k
            w_k  = np.sum(B * (W[:, n, :] + W[:, :, n]), axis = 1)

            # 3. determine interim allocation A
            A[n] = np.argmin(w_k)

            # 4. determine interim allocation B based on A
            B[:, :] = 0 # reset allocations
            B[A, np.arange(N)] = 1
            
            # 5. compute sum weighted interference
            mask = B[A, :]
            F = np.sum(W[A[:, None], np.arange(N)[:, None], np.where(mask == 1)[1]])            
            total_interference.append(F)
    
    return A, np.array(total_interference)

```

[5.49260555 0.         0.         0.        ]
[1.07034292 0.         0.         0.        ]
[2.29521014 0.         0.         0.        ]
[49.85659328  0.          0.          0.        ]
[0.39088826 0.         0.         0.        ]
[0.70537804 0.         0.         0.        ]
[3.82013518 0.         0.         0.        ]
[0.68353751 0.         0.         0.        ]
[0.44423408 0.         0.         0.        ]
[0.70218191 0.         0.         0.        ]
[1.47015893 0.         0.         0.        ]
[0.85038697 0.         0.         0.        ]
[1.00682202 0.         0.         0.        ]

```